In [ ]:
import random
import pandas as pd
import numpy as np

class HexGame:
    def __init__(self, BOARD_DIM):
        self.BOARD_DIM = BOARD_DIM
        self.neighbors = [-(self.BOARD_DIM + 2) + 1, -(self.BOARD_DIM + 2), -1, 1, (self.BOARD_DIM + 2), (self.BOARD_DIM + 2) - 1]
        self.board = [0] * ((self.BOARD_DIM + 2) * (self.BOARD_DIM + 2) * 2)
        self.open_positions = [0] * (self.BOARD_DIM * self.BOARD_DIM)
        self.number_of_open_positions = self.BOARD_DIM * self.BOARD_DIM
        self.moves = [0] * (self.BOARD_DIM * self.BOARD_DIM)
        self.connected = [0] * ((self.BOARD_DIM + 2) * (self.BOARD_DIM + 2) * 2)
        self.moves_player_X = 0  
        self.moves_player_O = 0 
        self.starting_player = random.choice([0, 1]) 
        self.init_game()


    def place_piece_randomly(self, player):
        random_empty_position_index = random.randint(0, self.number_of_open_positions - 1)
        empty_position = self.open_positions[random_empty_position_index]
        self.board[empty_position * 2 + player] = 1
        self.moves[self.BOARD_DIM * self.BOARD_DIM - self.number_of_open_positions] = empty_position
        self.open_positions[random_empty_position_index] = self.open_positions[self.number_of_open_positions - 1]
        self.number_of_open_positions -= 1


        if player == 0:
            self.moves_player_X += 1
        else:
            self.moves_player_O += 1

        return empty_position

    
    def init_game(self):
        for i in range(self.BOARD_DIM + 2):
            for j in range(self.BOARD_DIM + 2):
                self.board[(i * (self.BOARD_DIM + 2) + j) * 2] = 0
                self.board[(i * (self.BOARD_DIM + 2) + j) * 2 + 1] = 0
                if 0 < i < self.BOARD_DIM + 1 and 0 < j < self.BOARD_DIM + 1:
                    self.open_positions[(i - 1) * self.BOARD_DIM + (j - 1)] = i * (self.BOARD_DIM + 2) + j
                if i == 0:
                    self.connected[(i * (self.BOARD_DIM + 2) + j) * 2] = 1
                else:
                    self.connected[(i * (self.BOARD_DIM + 2) + j) * 2] = 0
                if j == 0:
                    self.connected[(i * (self.BOARD_DIM + 2) + j) * 2 + 1] = 1
                else:
                    self.connected[(i * (self.BOARD_DIM + 2) + j) * 2 + 1] = 0
    
    def connect(self, player, position):
        self.connected[position * 2 + player] = 1
        if player == 0 and position // (self.BOARD_DIM + 2) == self.BOARD_DIM:
            return True  
        if player == 1 and position % (self.BOARD_DIM + 2) == self.BOARD_DIM:
            return True  
        for i in range(6):
            neighbor = position + self.neighbors[i]
            if self.board[neighbor * 2 + player] and not self.connected[neighbor * 2 + player]:
                if self.connect(player, neighbor):
                    return True
        return False
    
    def check_winner(self, player, position):
        for i in range(6):
            neighbor = position + self.neighbors[i]
            if self.connected[neighbor * 2 + player]:
                return self.connect(player, position)
        return False
    
    def full_board(self):
        return self.number_of_open_positions == 0
    
    def get_board_state(self):
        board_state = ""
        for i in range(self.BOARD_DIM):
            for j in range(self.BOARD_DIM):
                pos = ((i + 1) * (self.BOARD_DIM + 2) + j + 1) * 2
                if self.board[pos] == 1:
                    board_state += "X"
                elif self.board[pos + 1] == 1:
                    board_state += "O" 
                else:
                    board_state += " "  
        return board_state  



def generate_hex_game_dataset(BOARD_DIM, num_games):
    X_data = []
    y_data = []
    starting_players = []
    no_cells = BOARD_DIM * BOARD_DIM
    move_order = []

    for game in range(num_games):
        hg = HexGame(BOARD_DIM)
        player = hg.starting_player
        starting_players.append(player) 
        winner = -1

        while not hg.full_board():
            position = hg.place_piece_randomly(player)
            row = (position // (BOARD_DIM + 2)) - 1
            col = (position % (BOARD_DIM + 2)) - 1
            move_order.append([player, row,col])


            """if 0 <= row < BOARD_DIM and 0 <= col < BOARD_DIM:
                print(f"Player {player}, placed a piece at position ({row}, {col})")
            else:
                print(f"Invalid position ({row}, {col}) for Player {player}. Check HexGame implementation.")
"""
            if hg.check_winner(player, position):
                winner = player
                break
            player = 1 - player

        X_data.append(hg.get_board_state())
        y_data.append(winner)

    return X_data, y_data, move_order



def print_hex_board(board_state):
   
    if len(board_state) < BOARD_DIM * BOARD_DIM:
        board_state = board_state.ljust(BOARD_DIM * BOARD_DIM)
    elif len(board_state) > BOARD_DIM * BOARD_DIM:
        board_state = board_state[:BOARD_DIM * BOARD_DIM]
    
  
    board_array = np.array(list(board_state)).reshape(BOARD_DIM, BOARD_DIM)
    
    for i in range(BOARD_DIM):
        print(" " * i, end="")
        
        for j in range(BOARD_DIM):
            cell = board_array[i, j]
            if cell == ' ':
                print(". ", end="")  
            else:
                print(f"{cell} ", end="")  
        print() 

def convert_to_dataframe(X_data, y_data, board_dim):
    columns = [f"cell{i}_{j}" for i in range(board_dim) for j in range(board_dim)] + ["winner"]
    data = []
    
    for board_state, winner in zip(X_data, y_data):
        row = []
        index = 0
        for i in range(board_dim):
            for j in range(board_dim):
                cell = board_state[index]
                if cell == "X":
                    row.append(1)  
                elif cell == "O":
                    row.append(0)  
                else:
                    row.append(-1) 
                index += 1
        row.append(winner)
        data.append(row)
    
    return pd.DataFrame(data, columns=columns)

In [ ]:
import argparse
import numpy as np
import pandas as pd
from time import time
import random
from sklearn.model_selection import train_test_split

from GraphTsetlinMachine.graphs import Graphs
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine

import pandas as pd

def convert_to_matrix(X,y, board_size):
    X_matrix = X.apply(lambda row: np.array(row.values).reshape((board_size, board_size)),axis=1)
    return X_matrix, y

def load_hex_game_df(data):
    

    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]
    player_mapping = {
        0: 'O', 
         1: 'X',  
         -1: ' '   
    }

    X = X.applymap(player_mapping.get)  
    y = y.map(player_mapping.get)     

    return X, y
    


def default_args():
    args = {
        "epochs": 50,
        "number_of_clauses": 10000, # 6000, 10 000
        "T": 8000,  # 4800, 8000
        "s": 1,
        #"number_of_state_bits": 8,
        "depth": 4,
        "hypervector_size": 256,#256
        "hypervector_bits": 2,
        "message_size": 256, #256
        "message_bits": 2,
        "double_hashing": False,
        "noise": 0.01,
        "number_of_examples": 3000,
        "number_of_classes": 2,
        #"max_sequence_length": 10,
        "max_included_literals": 32
    }
    return args


In [ ]:

def get_number_of_neighbors(board_size, row, col):
    if row == 0 and col == 0:
        return 2
    elif row == board_size - 1 and col == board_size - 1:
        return 2
    elif row == 0 and col == board_size - 1:
        return 3
    elif row == board_size - 1 and col == 0:
        return 3
    elif row == 0 or row == board_size - 1:
        return 4
    elif col == 0 or col == board_size - 1:
        return 4
    else:
        return 6

        
def get_neighbors_positions(board_size, row, col):
    upper_left = (0,0)
    bottom_right = (board_size-1, board_size-1)
    upper_right = (0, board_size-1)
    bottom_left = (board_size-1, 0)
    
    corners = [upper_left, bottom_right,upper_right, bottom_left]
    
    neighbors = []
    #upper left
    if (row, col) == corners[0]: 
        neighbors.extend([(row +1, col), (row, col+1)] )
    #bottom right
    elif (row, col) == corners[1]:
        neighbors.extend([(row, col -1),(row -1,col )] )
    #upper right
    elif (row, col) == corners[2]: 
        neighbors.extend([(row, col -1),(row +1,col),(row +1,col -1)])
    #bottom left
    elif (row, col) == corners[3]: 
        neighbors.extend([(row -1,col),(row -1, col +1),(row, col +1)])


    elif row == 0:
        neighbors.extend([(row,col-1),(row, col+1),(row+1,col-1),(row+1, col)])
    elif row == board_size-1:
        neighbors.extend([(row-1,col),(row -1,col + 1),(row,col + 1),(row,col -1)])

    elif col == 0:
        neighbors.extend([(row -1,col),(row -1,col + 1),(row,col + 1),(row+1, col)])
    elif col == board_size-1:
        neighbors.extend([(row -1,col ),(row,col - 1),(row +1,col - 1), (row +1,col)])
    else:
        neighbors.extend([(row,col +1),(row,col - 1),(row+1,col), (row-1, col),(row-1, col+1),(row+1, col-1)])

    return neighbors


def flatten_boards(X_matrix, y):
    flattened_data = []
    for board, winner in zip(X_matrix, y):
        
        flattened_board = ''.join(cell for row in board for cell in row)
        flattened_data.append({'board': flattened_board, 'winner': winner})
    return pd.DataFrame(flattened_data)

def flatten_to_matrix(flat_board, board_size):
    return [flat_board[i * board_size:(i + 1) * board_size] for i in range(board_size)]


def get_row_col(index, board_size):
    
    row = index // board_size
    col = index % board_size
    return row, col


def get_node_id(row, col, board_size):
   
    return row * board_size + col

def map_to_symbol(values):
    return [1 if value == 'X' else 0 for value in values]

def remap_to_symbol(values):
    return ['X' if value == 1 else 'O' for value in values]

def reshape_board(flattened_board, board_size):
    return np.array(list(flattened_board)).reshape((board_size, board_size))
    


In [ ]:
def neighbor_connectivity(board_size, grid, symbol, row, col):
    "Connectivity: ratio of neighbors with same symbol to the total number of neighbors "
    neighbors = get_neighbors_positions(board_size, row, col)
    
    same_symbol_count = sum(1 for neighbor in neighbors if grid[neighbor[0], neighbor[1]] == symbol)
    
    total_neighbors = len(neighbors)
    
    if total_neighbors > 0:
        return same_symbol_count / total_neighbors
    else:
        return 0  


def count_neighbors_with_same_symbol(board_game, row, col, symbol):
    """
    Num of neighrbors with same symbol.
    """
    neighbors = get_neighbors_positions(len(board_game), row, col)
    count = 0
    for neighbor_row, neighbor_col in neighbors:
        if board_game[neighbor_row][neighbor_col] == symbol:
            count += 1
    return count



def dfs(board_game, row, col, symbol, visited):
   
    if row < 0 or col < 0 or row >= len(board_game) or col >= len(board_game[0]) or (row, col) in visited or board_game[row][col] != symbol:
        return 0

    visited.add((row, col))

    path_length = 1

    neighbors = get_neighbors_positions(len(board_game), row, col)
    for neighbor_row, neighbor_col in neighbors:
        path_length = max(path_length, 1 + dfs(board_game, neighbor_row, neighbor_col, symbol, visited))

    return path_length

def find_longest_connected_path(board_game, row, col, symbol):
    """
    longest connected path of the same symbol using dfs.
    """
    visited = set()
    return dfs(board_game, row, col, symbol, visited)


In [ ]:
def get_symbols(board_size, board_game):
    """
    features to be added
    """
    symbol_names = ["O", "X", " "]
 

    for row in range(board_size):
        for col in range(board_size):
            symbol_names.append(f"r:{row}")
            symbol_names.append(f"c:{col}")
            #symbol_names.append(f"{row}:{col}")

    # Neighbor ratio
    for i in range(11):  
        decimal_value = i / 10  
        symbol_names.append(f"n_r:{decimal_value:.1f}")
    
    # neighbors length with same symbol
    """for value in range(11):  
        symbol_names.append(f"length:{value}")
"""
    # longest connected path: dfs
    """max_path_length = 30  
    for value in range(max_path_length + 1): 
        symbol_names.append(f"longest_connected_path:{value}")
    """
    
    symbol_names = list(set(symbol_names))
    return symbol_names

In [ ]:
BOARD_DIM = 7
N_GAMES = 10000
X_data, y_data,move_order = generate_hex_game_dataset(BOARD_DIM, num_games=N_GAMES) 

df = convert_to_dataframe(X_data, y_data, BOARD_DIM)

X, y = load_hex_game_df(df)

X, y = convert_to_matrix(X, y, BOARD_DIM)

flattened_df = flatten_boards(X, y)

print(flattened_df.head())

X = flattened_df['board']
y = flattened_df['winner']


X = np.array(X)
y = np.array(y)

symbol_names = get_symbols(board_size, X)




In [ ]:

args = default_args()

subset_size = int(len(X) * 0.7)
X_train = X[:subset_size]
X_test = X[subset_size:]
y_train = y[:subset_size]
y_test = y[subset_size:]


X_train = np.array(X_train)

X_test = np.array(X_test)



y_train = map_to_symbol(y_train)
y_train = np.array(y_train)

y_test = map_to_symbol(y_test)
y_test = np.array(y_test)

move_order_train = move_order[:subset_size]
move_order_test = move_order[subset_size:]






In [ ]:
print("Creating training data")
print("number of training graphs", X_train.shape[0])

def create_train_graph(X_train):

    graphs_train = Graphs(
       number_of_graphs=X_train.shape[0],
       symbols=symbol_names,
       hypervector_size=args["hypervector_size"],  
       hypervector_bits=args["hypervector_bits"],  
       double_hashing=args["double_hashing"],      
    )
    
    for graph_id in range(X_train.shape[0]):  
        graphs_train.set_number_of_graph_nodes(
            graph_id=graph_id,
            number_of_graph_nodes=board_dim,
        )
    graphs_train.prepare_node_configuration()
    
    for graph_id in range(X_train.shape[0]): 
        for node_id in range(board_dim):
            
            row,col = get_row_col(node_id, board_size)
        
            num_edges = get_number_of_neighbors(board_size, row, col)
        
            graphs_train.add_graph_node(graph_id, node_id, num_edges)
            
    
    graphs_train.prepare_edge_configuration()  
    
    # Connect the nodes
    for graph_id in range(X_train.shape[0]):
        visited_x = set()
        visited_o = set()
        visited = set()
        board_game = flatten_to_matrix(X_train[graph_id], board_size)
        reshaped_grid = reshape_board(X_train[graph_id], board_size)
        for node_id in range(board_dim):
            current_node = X_train[graph_id][node_id]
            

            row_pos,col_pos = get_row_col(node_id, board_size)
            node_pos = f"{row_pos}:{col_pos}"


            neighbor_ratio = neighbor_connectivity(board_size, reshaped_grid, current_node, row_pos, col_pos)
            neighbor_ratio = round(neighbor_ratio, 1)
            
            neighbor_length = count_neighbors_with_same_symbol(board_game, row_pos, col_pos, current_node)

            graphs_train.add_graph_node_property(graph_id, node_id, current_node)
            #graphs_train.add_graph_node_property(graph_id, node_id, node_pos)
           
            graphs_train.add_graph_node_property(graph_id, node_id, f"r:{row_pos}")
            graphs_train.add_graph_node_property(graph_id, node_id, f"c:{col_pos}")
            
    
            graphs_train.add_graph_node_property(graph_id, node_id, f"n_r:{neighbor_ratio}")
        
            #graphs_train.add_graph_node_property(graph_id, node_id, f"length:{neighbor_length}")
        
            """if current_node != ' ':
                longest_path = find_longest_connected_path(board_game, row_pos, col_pos, current_node)
            else:
                longest_path = 0  
                
            graphs_train.add_graph_node_property(graph_id, node_id, f"longest_connected_path:{longest_path}")
            """
            
         
           
         
    for graph_id in range(X_train.shape[0]):
        edge_type = "Plain"
        
        for node_id in range(board_dim):
            current_node = X_train[graph_id][node_id]
            row,col = get_row_col(node_id, board_size)
            
            neighbors = get_neighbors_positions(board_size, row, col)
           
            neighbor_ids = [get_node_id(n_row, n_col, board_size) for n_row, n_col in neighbors]
           
            for neighbor_id in neighbor_ids:
                graphs_train.add_graph_node_edge(graph_id, node_id, neighbor_id, edge_type)
                
        
    graphs_train.encode()
    
    return graphs_train



In [ ]:
print("Creating test data")
print("number of test graphs", X_test.shape[0])

def create_test_graph(X_test):

    graphs_test = Graphs(X_test.shape[0], init_with=graphs_train)
    
    for graph_id in range(X_test.shape[0]):  
        graphs_test.set_number_of_graph_nodes(
            graph_id=graph_id,
            number_of_graph_nodes=board_dim,
        )
    graphs_test.prepare_node_configuration()
    
    for graph_id in range(X_test.shape[0]): 
        for node_id in range(board_dim):
            
            row,col = get_row_col(node_id, board_size)
        
            num_edges = get_number_of_neighbors(board_size, row, col)
        
            graphs_test.add_graph_node(graph_id, node_id, num_edges)
            
    
    graphs_test.prepare_edge_configuration()  
    
    # Connect the nodes
    for graph_id in range(X_test.shape[0]):
        visited_x = set()
        visited_o = set()
        visited = set()
        board_game = flatten_to_matrix(X_test[graph_id], board_size)
        reshaped_grid = reshape_board(X_test[graph_id], board_size)
        for node_id in range(board_dim):
            current_node = X_test[graph_id][node_id]
            
         
            row_pos,col_pos = get_row_col(node_id, board_size)
            node_pos = f"{row_pos}:{col_pos}"
            
            neighbor_ratio = neighbor_connectivity(board_size, reshaped_grid, current_node, row_pos, col_pos)
            
            neighbor_ratio = round(neighbor_ratio, 1)
            
            neighbor_length = count_neighbors_with_same_symbol(board_game, row_pos, col_pos, current_node)

           
            graphs_test.add_graph_node_property(graph_id, node_id, current_node)
            #graphs_test.add_graph_node_property(graph_id, node_id, node_pos)
           
            graphs_test.add_graph_node_property(graph_id, node_id, f"r:{row_pos}")
            graphs_test.add_graph_node_property(graph_id, node_id, f"c:{col_pos}")
            graphs_test.add_graph_node_property(graph_id, node_id, f"n_r:{neighbor_ratio}")
            
            
            #graphs_test.add_graph_node_property(graph_id, node_id, f"length:{neighbor_length}")
            
            """if current_node != ' ':
                longest_path = find_longest_connected_path(board_game, row_pos, col_pos, current_node)
            else:
                longest_path = 0
                
            graphs_test.add_graph_node_property(graph_id, node_id, f"longest_connected_path:{longest_path}")
            """
        
         
    for graph_id in range(X_test.shape[0]):
        edge_type = "Plain"
        
        for node_id in range(board_dim):
            current_node = X_test[graph_id][node_id]
            row,col = get_row_col(node_id, board_size)
            
            neighbors = get_neighbors_positions(board_size, row, col)
           
            neighbor_ids = [get_node_id(n_row, n_col, board_size) for n_row, n_col in neighbors]
            
            for neighbor_id in neighbor_ids:
                graphs_test.add_graph_node_edge(graph_id, node_id, neighbor_id, edge_type)
                
        
    graphs_test.encode()
    
    return graphs_test



In [ ]:
graphs_train = create_train_graph(X_train)
graphs_test = create_test_graph(X_test)

tm = MultiClassGraphTsetlinMachine(
    args["number_of_clauses"],
    args["T"],
    args["s"],
    depth=args["depth"],
    message_size=args["message_size"],
    message_bits=args["message_bits"],
    max_included_literals=args["max_included_literals"],
    grid=(16*13,1,1),
    block=(128,1,1) 
)

start_training = time()
for i in range(args['epochs']):
    tm.fit(graphs_train, y_train, epochs=1, incremental=True) 
  
    train_accuracy = np.mean(y_train == tm.predict(graphs_train))  
    test_accuracy = np.mean(y_test == tm.predict(graphs_test))  
    
    print(f"Epoch#{i+1} -- Accuracy train: {train_accuracy:.2f}", end=' ')
    print(f"-- Accuracy test: {test_accuracy:.2f} ")
stop_training = time()
print(f"Training completed in {stop_training - start_training} seconds.")


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(true_values, predicted_values):
    
    true_numeric = [1 if value == 'X' else 0 for value in true_values]
    predicted_numeric = [1 if value == 'X' else 0 for value in predicted_values]
    
    precision = precision_score(true_numeric, predicted_numeric)
    recall = recall_score(true_numeric, predicted_numeric)
    f1 = f1_score(true_numeric, predicted_numeric)
    
    return precision, recall, f1




In [ ]:
from sklearn.metrics import classification_report

def calculate_classwise_metrics(true_values, predicted_values):
    
    true_numeric = [1 if value == 'X' else 0 for value in true_values]
    predicted_numeric = [1 if value == 'X' else 0 for value in predicted_values]
    
   
    report = classification_report(true_numeric, predicted_numeric, target_names=['O', 'X'])
    
    return report



In [ ]:
import random

def remove_last_k_moves(flattened_board, k, move_order, board_size):
    
    flattened_board = list(flattened_board)
    last_k_moves = move_order[-k:] 
    
    for move in last_k_moves:
        player, row, col = move
        index_to_remove = row * board_size + col
        flattened_board[index_to_remove] = ' '  

    return ''.join(flattened_board)

def create_modified_test_data(X_test, move_order, k, board_size):
    modified_X_test = []
    for board_game in X_test:
        modified_game = remove_last_k_moves(board_game, k, move_order, board_size)
        modified_X_test.append(modified_game)
    
    return modified_X_test






## Performance Metrics - all moves

In [ ]:
predictions = tm.predict(graphs_test) 
true_values = remap_to_symbol(y_test) 
predicted_values = remap_to_symbol(predictions)  


precision, recall, f1 = calculate_metrics(true_values, predicted_values)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Class wise
predictions = tm.predict(graphs_test)  
true_values = remap_to_symbol(y_test) 
predicted_values = remap_to_symbol(predictions)  


report = calculate_classwise_metrics(true_values, predicted_values)


print(report)



## Performance metrics - two moves before

In [ ]:
k = 2  

X_test_modified = create_modified_test_data(X_test, move_order_test, k, board_size=7)
X_test_modified = np.array(X_test_modified)


graphs_test_two_moves = create_test_graph(X_test_modified)
test_accuracy_two_moves = np.mean(y_test == tm.predict(graphs_test_two_moves))
print(f"Two moves before end: {test_accuracy_two_moves:.2f}")

predictions_two_moves = tm.predict(graphs_test_two_moves)

true_values = remap_to_symbol(y_test) 
predicted_values_two = remap_to_symbol(predictions_two_moves) 
precision, recall, f1 = calculate_metrics(true_values, predicted_values_two)


print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

report = calculate_classwise_metrics(true_values, predicted_values_two)
print(report)


## Performance metrics - Five moves before

In [ ]:
k = 5  

X_test_modified_five = create_modified_test_data(X_test, move_order_test, k, board_size=7)
X_test_modified_five = np.array(X_test_modified_five)

graphs_test_five_moves = create_test_graph(X_test_modified_five)
test_accuracy_five_moves = np.mean(y_test == tm.predict(graphs_test_five_moves))
print(f"Five moves before end: {test_accuracy_five_moves:.2f}")

predictions_five_moves = tm.predict(graphs_test_five_moves)

true_values = remap_to_symbol(y_test) 
predicted_values_five = remap_to_symbol(predictions_five_moves) 
precision, recall, f1 = calculate_metrics(true_values, predicted_values_five)


print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

report = calculate_classwise_metrics(true_values, predicted_values_five)
print(report)


## Visualization and analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import collections
import math
from random import randint

Point = collections.namedtuple("Point", ["x", "y"])
_Hex = collections.namedtuple("Hex", ["q", "r", "s"])

def Hex(q, r, s):
    assert not (round(q + r + s) != 0), "q + r + s must be 0"
    return _Hex(q, r, s)

Orientation = collections.namedtuple("Orientation", ["f0", "f1", "f2", "f3", "b0", "b1", "b2", "b3", "start_angle"])
Layout = collections.namedtuple("Layout", ["orientation", "size", "origin"])

layout_pointy = Orientation(math.sqrt(3.0), math.sqrt(3.0) / 2.0, 0.0, 3.0 / 2.0, math.sqrt(3.0) / 3.0, -1.0 / 3.0, 0.0, 2.0 / 3.0, 0.5)

class HexBoard:
    def __init__(self, N=7, hex_size=10, origin=(0,50), fig=None, ax=None, winner=None):
        self.N = N  
        self.layout = Layout(layout_pointy, Point(hex_size, hex_size), Point(*origin))
        if fig is None or ax is None:
            self.fig, self.ax = plt.subplots()
        else:
            self.fig, self.ax = fig, ax
        
        self.ax.set_aspect('equal')
        self.winner = winner
        self.winner_symbol = "X" if self.winner == 1 else "O"


    def hex_to_pixel(self, h):
        M = self.layout.orientation
        size = self.layout.size
        origin = self.layout.origin
        x = (M.f0 * h.q + M.f1 * h.r) * size.x
        y = (M.f2 * h.q + M.f3 * h.r) * size.y
        pixel_point = Point(x + origin.x, y + origin.y)
        return pixel_point


    def hex_corner_offset(self, corner):
        M = self.layout.orientation
        size = self.layout.size
        angle = 2.0 * math.pi * (M.start_angle - corner) / 6.0
        return Point(size.x * math.cos(angle), size.y * math.sin(angle))

    def polygon_corners(self, h):
        corners = []
        center = self.hex_to_pixel(h)
        for i in range(0, 6):
            offset = self.hex_corner_offset(i)
            corners.append(Point(center.x + offset.x, center.y + offset.y))
        return corners

    def draw_hex(self, h, symbol='', color='white'):
        corners = self.polygon_corners(h)
        hex_polygon = Polygon(
            [(p.x, p.y) for p in corners], closed=True, edgecolor='black', facecolor=color, lw=1
        )
        self.ax.add_patch(hex_polygon)

        center = self.hex_to_pixel(h)
        if symbol:
            self.ax.text(center.x, center.y + 5, symbol, ha='center', va='center', fontsize=10, weight='bold', color='red' if symbol == 'X' else 'blue')
        

    def get_color(self, value):
        if value == 1:
            return 'red'
        elif value == -1:
            return 'blue'
        else:
            return 'white'

    def plot_empty_board(self):
        for r in range(self.N):
            for q in range(self.N-1,-1,-1):
                h = Hex(q, r, -q - r)
                self.draw_hex(h, color='white')
              
    def populate_board(self, sample):
        for r in range(self.N):
            for q in range(self.N):
                cell_value = sample.get(f'cell{r}_{q}', 0)  
                symbol = 'X' if cell_value == 1 else 'O' if cell_value == -1 else ' '

                h = Hex(q, r, -q - r)
                self.draw_hex(h, symbol=symbol, color='white')  
                

    def populate_critical_hex(self, critical_positions,player_symbol, cmap):
        max_count = max(critical_positions.values())
        

        for r in range(self.N):
            for q in range(self.N):
                h = Hex(q, r, -q - r)
                mirrored_r = self.N - 1 - r
                pos = (mirrored_r, q)

                if pos in critical_positions:
                    color = cmap(critical_positions[pos] / max_count)
                    self.draw_critical_hex(h, color=color, alpha=0.9, label=f"{critical_positions[pos]}")
                else:
                    self.draw_hex(h, color='white')
        plt.title(f"Positions part of winning path for'{player_symbol}'", fontsize=15, weight='bold')

    def draw_critical_hex(self, h, color='white', edge_color='black', alpha=1.0, label=''):
        corners = self.polygon_corners(h)
        hex_polygon = Polygon([(p.x, p.y) for p in corners], closed=True,
                              edgecolor=edge_color, facecolor=color, lw=1, alpha=alpha)
        self.ax.add_patch(hex_polygon)
        if label:
            center = self.hex_to_pixel(h)
            self.ax.text(center.x, center.y, label, ha='center', va='center',
                         fontsize=8, weight='bold', color='black')
    
    def add_labels(self, winner):
        winner_text = "Player X wins!" if winner == 1 else "Player O wins!"
        self.ax.text(-240, 90, winner_text, fontsize=15, color='black', weight='bold')

        self.ax.text(-240, 75, "Player X (Red)", fontsize=12, color='red', weight='bold')
        self.ax.text(-240, 60, "Player O (Blue)", fontsize=12, color='blue', weight='bold')
    def add_text(self, text):
        self.ax.text(-240, 90, text, fontsize=15, color='black', weight='bold')
        

    def set_plot_limits(self, xlim=(-250, 100), ylim=(-250, 50)):
        self.ax.set_xlim(*xlim)  
        self.ax.set_ylim(*ylim)  
        plt.axis('off')
    def highlight_winning_path(self, moves, winning_path, player_symbol, path_color='green'):

        for r in range(self.N):
            for q in range(self.N):
                h = Hex(q, r, -q - r)

                if (r, q) in winning_path:
                    self.draw_hex(h, symbol=player_symbol, color=path_color)
                else:
                    cell_value = moves[r, q]
                    if cell_value == 'X':  # Spiller X
                        self.draw_hex(h, symbol='X', color='white')
                    elif cell_value == 'O':  # Spiller O
                        self.draw_hex(h, symbol='O', color='white')
                    else:  # Tom celle
                        self.draw_hex(h, symbol=' ', color='white')

        plt.title(f"Winning Path for '{player_symbol}'", fontsize=15, weight='bold')


    def show(self):
        """Display the plot."""
        plt.show()


In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

def build_hex_graph(board_size, moves, player_symbol):
    """Graph for one player only"""
    G = nx.Graph()
    neighbors = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, 1), (1, -1)]  

    for row, col in np.argwhere(moves == player_symbol): 
        G.add_node((row, col)) 
        for dr, dc in neighbors:
            nr, nc = row + dr, col + dc
            if 0 <= nr < board_size and 0 <= nc < board_size and moves[nr, nc] == player_symbol:
                G.add_edge((row, col), (nr, nc))  
    return G


def find_winning_path(G, board_size, player_symbol):
    """Winning path for a player"""
    if player_symbol == 'X':  
        start_nodes = [(r, 0) for r in range(board_size)]  
        end_nodes = [(r, board_size-1) for r in range(board_size)] 
    else:  
        start_nodes = [(0, c) for c in range(board_size)]  
        end_nodes = [(board_size-1, c) for c in range(board_size)]  
    
    valid_start_nodes = [node for node in start_nodes if node in G.nodes()]
    valid_end_nodes = [node for node in end_nodes if node in G.nodes()]
    
    
    for start in valid_start_nodes:
        for end in valid_end_nodes:
            if nx.has_path(G, start, end):
                return nx.shortest_path(G, start, end)
    return []


def plot_winning_path(board_size, moves, winning_path, player_symbol):
    plt.figure(figsize=(8, 8))
    
    for row in range(board_size):
        for col in range(board_size):
            offset = row * 0.5  
            color = 'white'
            if moves[row, col] == 'X':
                color = 'red'
            elif moves[row, col] == 'O':
                color = 'blue'
            plt.scatter(col + offset, -row, s=500, c=color, edgecolor='black')

    if winning_path:
        x_coords = [node[1] + node[0] * 0.5 for node in winning_path]
        y_coords = [-node[0] for node in winning_path]
        plt.scatter(x_coords, y_coords, s=500, c='green', edgecolor='black', label=f"Winning Path of '{player_symbol}'")

    plt.title("Winning Path")
    plt.xlabel("Column")
    plt.ylabel("Row")
    plt.legend(loc="upper right") 
    plt.axis('off')
    plt.show()

def parse_game(game_string, board_size):
    moves = []
    for index, symbol in enumerate(game_string):
        if symbol.strip():  # Hopp over tomme celler
            row = index // board_size
            col = index % board_size
            moves.append({'row': row, 'col': col, 'symbol': symbol})
    return moves


def extract_moves(game, board_size=7):
    moves = np.full((board_size, board_size), ' ')
    print("Game structure:", game)
    print("Extracted moves:", moves)
    for cell in game:
        row, col = int(cell['row']), int(cell['col'])
        print(row,col)
        moves[row, col] = cell['symbol']  
    

    return moves

def analyze_critical_positions(test_set, board_size=7, player_symbol='X'):
    critical_positions = defaultdict(int)

    for game in test_set:
        print(f"Original game string: {game}")
        parsed_game = parse_game(game, board_size)
        print(f"Parsed game moves: {parsed_game}")

        moves = extract_moves(parsed_game, board_size)
        G = build_hex_graph(board_size, moves, player_symbol)
        winning_path = find_winning_path(G, board_size, player_symbol)

        for node in winning_path:
            critical_positions[node] += 1  

    return critical_positions



In [ ]:
critical_positions_X = analyze_critical_positions(X_test, board_size=7, player_symbol='X')
critical_board_X  = HexBoard(N=7, hex_size=15, origin=(-200, -125))

cmap = plt.cm.Reds
critical_board_X.populate_critical_hex(critical_positions_X , player_symbol='X',cmap=cmap)
critical_board_X.set_plot_limits()
critical_board_X.show()


In [ ]:

critical_positions_O = analyze_critical_positions(X_test, board_size=7, player_symbol='O')
critical_board_O  = HexBoard(N=7, hex_size=15, origin=(-200, -125))
cmap = plt.cm.Blues
critical_board_O.populate_critical_hex(critical_positions_O , player_symbol='O',cmap=cmap)
critical_board_O.set_plot_limits()
critical_board_O.show()

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

def count_winner_moves(X_test, y_test, board_size=7):

    winner_moves = []
    
    for game, winner in zip(X_test, y_test):
        moves = extract_moves(game, board_size) 
        winner_symbol = 'X' if winner == 1 else 'O'  
        
        num_moves = np.sum(moves == winner_symbol)
        winner_moves.append(num_moves)
    
    return winner_moves

def visualize_winner_moves_distribution(winner_moves):
    
    move_counts = Counter(winner_moves)
    x_ticks = sorted(move_counts.keys())
    y_values = [move_counts[x] for x in x_ticks]
    
    plt.figure(figsize=(10, 6))
    plt.bar(x_ticks, y_values, color="skyblue", edgecolor="black", alpha=0.8)
    plt.xticks(x_ticks, rotation=45)
    plt.xlabel("Number of Moves")
    plt.ylabel("Number of Games")
    plt.title("Distribution of Moves by Winner")
    plt.tight_layout()
    plt.show()

winner_moves = count_winner_moves(X_test, y_test, board_size=7)
visualize_winner_moves_distribution(winner_moves)


In [ ]:
def convert_game_to_sample(game, sample_n=1, board_size=7):
    sample = {}
    game_data = game[sample_n]
    for cell in game_data:
        r = int(cell['row'])  
        c = int(cell['col'])  
        symbol = cell['symbol']  

        if symbol == 'X':
            sample[f'cell{r}_{c}'] = 1  # Player X
        elif symbol == 'O':
            sample[f'cell{r}_{c}'] = -1  # Player O
        else:
            sample[f'cell{r}_{c}'] = 0  
    return sample
game_data = parse_game(X_test[1], board_size=7)
sample1 = convert_game_to_sample([game_data], sample_n=0, board_size=7)

board = HexBoard(N=7, hex_size=15, origin=(-200, -125), winner=y_test[1])
board.populate_board(sample1)
board.add_labels(winner=y_test[1])
board.set_plot_limits()
board.show()

for r in range(7):
    for c in range(7):
        cell_key = f'cell{r}_{c}'
        print(f"{cell_key}: {sample1.get(cell_key, 0)}")
